In [1]:
import torch
import torch.nn.functional as F


In [1]:
class DL(torch.nn.Module):
    # how do you set the configure or dl architecture? 
    def __init__(self, n_inp, l_1, l_2, conv1_out, conv1_kernel, conv2_kernel, drop1 = 0, n_out = 1):
        super(DL, self).__init__()

        conv1_out_ch = conv1_out
        conv2_out_ch = conv1_out * 2
        conv1_kernel = conv1_kernel
        conv2_kernel = conv2_kernel
        self.dropout_lin1 = drop1
        self.pool = torch.nn.MaxPool1d(kernel_size = 2)
        self.conv1 = torch.nn.Conv1d(in_channels = 1, out_channels = conv1_out_ch, kernel_size = conv1_kernel, padding = conv1_kernel - 1)  # Configure of the model 
        self.conv2 = torch.nn.Conv1d(in_channels = conv1_out_ch, out_channels = conv2_out_ch, kernel_size = conv2_kernel, padding = conv2_kernel - 1)
        feature_tensor = self.feature_stack(torch.Tensor([[0] * n_inp]))
        
        # 
        self.lin1 = torch.nn.Linear(feature_tensor.size()[1], l_1)
        self.lin2 = torch.nn.Linear(l_1, l_2)
        self.lin3 = torch.nn.Linear(l_2, n_out)

    def feature_stack(self, x):
        """
        when you input a feature tensor, then you break down the tensor and put the matrix into one single array
        and the pool them with one to one with 2 size matrix and then put them into second relu matrix
        finally flatten it up to 2 dimensions 
        """
        x = x.unsqueeze(1)
        x = F.relu(self.pool(self.conv1(x)))
        x = F.relu(self.pool(self.conv2(x)))
        x = x.flatten(start_dim = 1)
        return x

    def fc_stack(self, x):
        x1 = F.dropout(F.relu(self.lin1(x)), p = self.dropout_lin1)  # input F-relu, dropout_lin1 = 0.1 - randomly zeroouts some of the elements 
        x2 = F.relu(self.lin2(x1))  # 
        y = self.lin3(x2)
        return y

    def forward(self, x):
        x = self.feature_stack(x)
        y = self.fc_stack(x)
        return y

NameError: name 'torch' is not defined

In [ ]:
net = DL(n_inp = 120, l_1 = 400, l_2 = 48, conv1_out = 6, conv1_kernel = 36, conv2_kernel = 12, drop1 = .1)